In [8]:
import os
from dotenv import load_dotenv
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import (PromptTemplate, 
                               SystemMessagePromptTemplate, 
                               ChatPromptTemplate,
                              HumanMessagePromptTemplate)
from langchain.llms import OpenAI

In [9]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key
model = ChatOpenAI(openai_api_key=api_key)

## Part 1

In [10]:
# Step 1: import Parser
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()
# Step 2 - format instructions
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [11]:
# Test
# reply = "red, blue, green"
# output_parser.parse(reply)

In [12]:
humman_template = "{request}\n{format_instructions}"
humman_prompt = HumanMessagePromptTemplate.from_template(humman_template)

In [13]:
chat_prompt = ChatPromptTemplate.from_messages([humman_prompt])

In [14]:
request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                         format_instructions=output_parser.get_format_instructions())
request

ChatPromptValue(messages=[HumanMessage(content='give me 5 characteristics of dogs\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')])

In [15]:
model_request = request.to_messages()
result = model.invoke(model_request)
result.content

'Loyal, playful, protective, social, intelligent'

## Part 2

In [16]:
from langchain.output_parsers import DatetimeOutputParser

In [17]:
output_parser = DatetimeOutputParser()
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1950-10-24T22:39:35.653343Z, 0836-09-13T08:31:20.348473Z, 1596-04-25T07:35:04.802673Z\n\nReturn ONLY this string, no other words!"

In [18]:
template_text = "{request}\n{format_instructions}"
humman_prompt = HumanMessagePromptTemplate.from_template(template_text)
chat_prompt = ChatPromptTemplate.from_messages([humman_prompt])

In [19]:
request = chat_prompt.format_prompt(
            request="What date was the 13th Amendment ratified in the US?",
            format_instructions = output_parser.get_format_instructions())
request 

ChatPromptValue(messages=[HumanMessage(content="What date was the 13th Amendment ratified in the US?\nWrite a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 0488-05-24T17:55:44.391840Z, 0109-05-30T02:28:16.156136Z, 1763-02-03T14:08:30.487244Z\n\nReturn ONLY this string, no other words!")])

In [20]:
model_request = request.to_messages()

In [21]:
result = model.invoke(model_request,temperature=0)
result.content

'1865-12-06T00:00:00.000000Z'

In [22]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

## Part 3

In [23]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [24]:
class Scientist(BaseModel):
    name: str = Field(description='Name of a Scientist')
    discoveries:list = Field(description='Python list of discoveries')

In [25]:
parser = PydanticOutputParser(pydantic_object=Scientist)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [27]:
humman_template = "{request}\n{format_instructions}"
humman_prompt = HumanMessagePromptTemplate.from_template(humman_template)
chat_prompt = ChatPromptTemplate.from_messages([humman_prompt])

In [28]:
request = chat_prompt.format_prompt(
            request="Tell me about a famous scientist",
            format_instructions = parser.get_format_instructions())
request 

ChatPromptValue(messages=[HumanMessage(content='Tell me about a famous scientist\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}\n```')])

In [29]:
model_request = request.to_messages()
result = model.invoke(model_request,temperature=0)
result.content

'{\n    "name": "Isaac Newton",\n    "discoveries": ["Law of Universal Gravitation", "Three Laws of Motion"]\n}'